Power spectrum for 1 sec pre- and 1 sec post- stimulus
not used in the final report

In [1]:
import database.neuralynx_extract.combinato_tools as ct
import database.neuralynx_extract.nlxio as nlxio

import preprocessing.data_preprocessing.refractor_utils as ru
import sys

import matplotlib.pyplot as plt
import datajoint as dj
import scipy
from pathlib import Path
import neurokit as nk
import neurokit2 as nk2

from statsmodels.stats import multitest
from numpy.random import RandomState
from permute.core import one_sample, two_sample
from netneurotools import stats
import mne

# Local application imports 
from database.db_setup import *
import preprocessing.data_preprocessing.binning as binning
import preprocessing.data_preprocessing.create_vectors_from_time_points as create

import utils.helper_func as hf
import utils.preprocessing as preprocessing

config


Please enter DataJoint username:  root
Please enter DataJoint password:  ···············


Connecting root@localhost:3306
Epiphyte Setup


In [10]:
top_dir = '/home/anastasia/epiphyte/anastasia/output'

patient_id = 53
session_nr = 1

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

times = np.linspace(-500, 1000, num = int(1.5*fs_downs))
time_zero_idx = np.where(times == hf.find_nearest(times, 0))[0][0]

df_stim_info = pd.DataFrame(ScreeningAnnotation())
df_stim_info = df_stim_info.loc[df_stim_info['patient_id'] == patient_id]
df_stim_info = df_stim_info.reset_index()
df_stim_info.to_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')
#get relation of filename to stim name
df = df_stim_info.drop_duplicates(subset=['filename'])
df.to_csv(f'{top_dir}/df_stim_filename.csv')

#get 500 events
df_movie = pd.DataFrame(MovieAnnotation())
df_movie.to_csv(f'{top_dir}/df_movie_annotaion.csv')

all_stim = np.unique(df_stim_info['stim_id'])

downs_dir = f'{top_dir}/01-preprocessed_{patient_id}/downsampled'
timestamps_downs = np.load(f'{downs_dir}/{patient_id}_timestamps_downs.npy')

Power spectra of the entire recording (~ 2 hours?) for each electrode separately.

In [9]:
#plot each channel separately
for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    my_file = Path(f'{downs_dir}/{patient_id}_channel-{ch}_downsampled_data.npy')
    downsampled_data = np.load(my_file)
    downsampled_data = preprocessing.notch_filter(downsampled_data, patient_id, ch)
    
    
    f, welchpow = scipy.signal.welch(downsampled_data,fs=fs_downs, nperseg=2000, noverlap=500)

    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 5))
    fig.suptitle(f'Channel: {ch}, recording site: {ch_site}')

    ax1.semilogy(f,welchpow)
    ax1.set_xlabel('frequency [Hz]')
    ax1.set_ylabel('Power [log]')

    ax2.loglog(f,welchpow)
    ax2.set_title('loglog scale')
    ax2.set_xlabel('frequency [Hz, log scale]')
    ax2.set_ylabel('Power [log]')
    
    fig.savefig(f'{top_dir}/03-stationary_FFT/entire_channel/{ch}_{ch_site}.png')
    plt.close()

In [10]:
alpha = 0.05
prng = RandomState(50)
n_permutations = 5000
method = 'fdr_bh'
columns = ['channel_name', 'channel_location', 'stim_index', 'stim_name', 'paradigm', 'corr_results', 'locations']

In [17]:
parent_dir = f'{top_dir}/03-stationary_FFT/post_post_comparison'
hf.create_folder_structure(parent_dir, df_patient_info)

Power spectrum computed separately for the epochs (2s) during pre-screening and post-screening.
I divided each epoch into the pre-stimulus (1s) and post-stimulus (1s) parts.

Each stimulus was presented 10 times. I computed the power spectrum for each presentation separately, then averaged the 10 resulting power spectra and plotted the averaged spectrum.

Then i computed t-test for non-averaged power spectra pre- and post-stimulus presentation.
Vertical lines on the plots indicate the frequencies at which the results were significant.

In [15]:
times = np.linspace(-1000, 1000, num = 2001)
list_strings_1 = []
list_strings_2 = []

for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    epochs = np.load(f'{top_dir}/02-epochs/broadband_2_sec/{patient_id}_epochs_channel-{ch}.npy')

    for st in all_stim:
        pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
        pre = pre.reset_index(drop=True)
        post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
        pre_index = np.array(pre['stim_index'])
        post_index = np.array(post['stim_index'])

        pre_pwr_array_1 = []
        pre_pwr_array_2 = []
        post_pwr_array_1 = []
        post_pwr_array_2 = []
        pre_epoch_1 = np.zeros((10,1000))
        pre_epoch_2 = np.zeros((10,1000))
        post_epoch_1 = np.zeros((10,1000))
        post_epoch_2 = np.zeros((10,1000))

        for i in range(len(pre_index)):
            pre_epoch = epochs[pre_index[i], :]
            post_epoch = epochs[post_index[i], :]

            pre_epoch_1[i,:] = pre_epoch[0:1000]
            pre_epoch_2[i,:] = pre_epoch[1001:2001]

            post_epoch_1[i,:] = post_epoch[0:1000]
            post_epoch_2[i,:] = post_epoch[1001:2001]

        N = len(pre_epoch_1[0,:])

        #check computation here
        pwr_pre_1 = np.abs(scipy.fftpack.fft(pre_epoch_1)/N)**2
        pwr_pre_2 = np.abs(scipy.fftpack.fft(pre_epoch_2)/N)**2
        pwr_post_1 = np.abs(scipy.fftpack.fft(post_epoch_1)/N)**2
        pwr_post_2 = np.abs(scipy.fftpack.fft(post_epoch_2)/N)**2

        freqs = scipy.fftpack.fftfreq(N) * fs_downs
        freqs = freqs[0:N//2]

        pwr_pre_1 = pwr_pre_1[:, 0:N//2]
        pwr_pre_2 = pwr_pre_2[:, 0:N//2]
        pwr_post_1 = pwr_post_1[:, 0:N//2]
        pwr_post_2 = pwr_post_2[:, 0:N//2]

        pre_pwr_array_1 = np.array(pre_pwr_array_1)
        pre_pwr_array_2 = np.array(pre_pwr_array_2)
        post_pwr_array_1 = np.array(post_pwr_array_1)
        post_pwr_array_2 = np.array(post_pwr_array_2)

        current_stim_index = pre.loc[0,'stim_index']
        current_stim_name = pre.loc[0,'stim_name']
        current_stim_paradigm = pre.loc[0,'paradigm']

        #t-test
        t_val, p_values = scipy.stats.ttest_rel(pwr_pre_1, pwr_pre_2, 0)
        output = multitest.multipletests(p_values, alpha=alpha, method=method)
        reject = output[0]
        pval_corrected = output[1]
        if True in reject:
            string = f'{method}-corrected significant results'
            #print(string)
            a1 = np.where(pval_corrected <= 0.05)[0]
            #print(a1)
            list_strings_1.append((ch, ch_site, current_stim_index, current_stim_name, current_stim_paradigm, string, a1))
        else:
            string = f"no {method}-corrected significant results"
            #list_strings.append(string)
            #print(f"no {method}-corrected significant results")
            a1=[]

        #t-test
        t_val, p_values = scipy.stats.ttest_rel(pwr_post_1, pwr_post_2, 0)
        output = multitest.multipletests(p_values, alpha=alpha, method=method)
        reject = output[0]
        pval_corrected = output[1]
        if True in reject:
            string = f'{method}-corrected significant results'
            #print(string)
            a2 = np.where(pval_corrected <= 0.05)[0]
            #print(a2)
            list_strings_2.append((ch, ch_site, current_stim_index, current_stim_name, current_stim_paradigm, string, a2))
        else:
            string = f"no {method}-corrected significant results"
            #list_strings.append(string)
            #print(f"no {method}-corrected significant results")
            a2=[]

        fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(20, 5))
        title = f'channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
        fig.suptitle(title)

        ax1.loglog(freqs,np.mean(pwr_pre_1, axis=0), label='pre')
        ax1.loglog(freqs,np.mean(pwr_pre_2, axis=0), label='post')
        ax1.set_title('pre movie')
        ax1.set_xlabel('Frequency [log]')
        ax1.set_ylabel('Power [log]')
        for k in range(int(len(a1))):
            plt.axvline(a1[k], color='k')
        ax1.legend()

        ax2.loglog(freqs,np.mean(pwr_post_1, axis=0), label='pre')
        ax2.loglog(freqs,np.mean(pwr_post_2, axis=0), label='post')
        ax2.set_title('post movie')
        ax2.set_xlabel('Frequency [log]')
        ax2.set_ylabel('Power [log]')
        for k in range(int(len(a2))):
            plt.axvline(a2[k], color='k')        
        ax2.legend()
        
        fig.savefig(f'{top_dir}/03-stationary_FFT/pre_post_comparison/{ch}_{ch_site}/{title}.png')
        plt.close()

        #plt.show()
        
df_pre_movie = pd.DataFrame(list_strings_1, columns=columns)
df_post_movie = pd.DataFrame(list_strings_2, columns=columns)

df_pre_movie.to_csv(f'{top_dir}/03-stationary_FFT/pre-movie.csv')
df_post_movie.to_csv(f'{top_dir}/03-stationary_FFT/post-movie.csv')


In [19]:
#post/post comparison
times = np.linspace(-1000, 1000, num = 2001)
list_strings_1 = []

for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    epochs = np.load(f'{top_dir}/02-epochs/broadband_2_sec/{patient_id}_epochs_channel-{ch}.npy')

    for st in all_stim:
        pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
        pre = pre.reset_index(drop=True)
        post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
        pre_index = np.array(pre['stim_index'])
        post_index = np.array(post['stim_index'])

        pre_pwr_array_1 = []
        pre_pwr_array_2 = []
        post_pwr_array_1 = []
        post_pwr_array_2 = []
        
        pre_epoch_1 = np.zeros((10,1000))
        pre_epoch_2 = np.zeros((10,1000))
        post_epoch_1 = np.zeros((10,1000))
        post_epoch_2 = np.zeros((10,1000))

        for i in range(len(pre_index)):
            pre_epoch = epochs[pre_index[i], :]
            post_epoch = epochs[post_index[i], :]

            #pre_epoch_1[i,:] = pre_epoch[0:1000]
            pre_epoch_2[i,:] = pre_epoch[1001:2001]

            #post_epoch_1[i,:] = post_epoch[0:1000]
            post_epoch_2[i,:] = post_epoch[1001:2001]

        N = len(pre_epoch_2[0,:])
        
        #pwr_pre_1 = np.abs(scipy.fftpack.fft(pre_epoch_1)/N)**2
        pwr_pre_2 = np.abs(scipy.fftpack.fft(pre_epoch_2)/N)**2
        #pwr_post_1 = np.abs(scipy.fftpack.fft(post_epoch_1)/N)**2
        pwr_post_2 = np.abs(scipy.fftpack.fft(post_epoch_2)/N)**2

        freqs = scipy.fftpack.fftfreq(N) * fs_downs
        freqs = freqs[0:N//2]

        #pwr_pre_1 = pwr_pre_1[:, 0:N//2]
        pwr_pre_2 = pwr_pre_2[:, 0:N//2]
        #pwr_post_1 = pwr_post_1[:, 0:N//2]
        pwr_post_2 = pwr_post_2[:, 0:N//2]

        pre_pwr_array_1 = np.array(pre_pwr_array_1)
        pre_pwr_array_2 = np.array(pre_pwr_array_2)
        post_pwr_array_1 = np.array(post_pwr_array_1)
        post_pwr_array_2 = np.array(post_pwr_array_2)
        
        current_stim_index = pre.loc[0,'stim_index']
        current_stim_name = pre.loc[0,'stim_name']
        current_stim_paradigm = pre.loc[0,'paradigm']        

        #t-test
        t_val, p_values = scipy.stats.ttest_rel(pwr_pre_2, pwr_post_2, 0)
        output = multitest.multipletests(p_values, alpha=alpha, method=method)
        reject = output[0]
        pval_corrected = output[1]
        if True in reject:
            string = f'{method}-corrected significant results'
            #list_strings.append(string)
            #print(string)
            a1 = np.where(pval_corrected <= 0.05)[0]
            #print(a1)
            list_strings_1.append((ch, ch_site, current_stim_index, current_stim_name, current_stim_paradigm, string, a1))
        else:
            string = f'no {method}-corrected significant results'
            #list_strings.append(string)
            #print(string)
            a1=[]
        
        plt.rcParams['figure.figsize'] = (20,3)
        plt.loglog(freqs,np.mean(pwr_pre_2, axis=0), label='pre_movie')
        plt.loglog(freqs,np.mean(pwr_post_2, axis=0), label='post_movie')
        for k in range(int(len(a1))):
            plt.axvline(a1[k], color='k')
        title = f'channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
        plt.title(title)
        plt.xlabel('Frequency')
        plt.ylabel('Power')
        plt.legend()
        
        #plt.show()
        
        plt.savefig(f'{top_dir}/03-stationary_FFT/post_post_comparison/{ch}_{ch_site}/{title}.png')
        plt.close()
        
        del a1

df_post_movie = pd.DataFrame(list_strings_1, columns=columns)
df_post_movie.to_csv(f'{top_dir}/03-stationary_FFT/post_post_comparison.csv')
